In [1]:
!pip install nlp
!pip install git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 1.7MB 2.8MB/s 
     |████████████████████████████████| 245kB 20.2MB/s 
     |████████████████████████████████| 17.3MB 198kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-26fp36x9
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-26fp36x9
     |████████████████████████████████| 3.0MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 31.8MB/s 
     |████████████████████████████████| 890kB 39.7MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=884139 sha256=0fbfda11e286dfb1b72679e997136c2c971de24c4a3d64c9eb29f0a3ec0e29b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhq698kd/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
  Created wheel for sacremoses: filename=sacremoses-0.0

In [2]:
from pathlib import Path
from copy import deepcopy

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F
from transformers import XLNetLMHeadModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from nlp import load_dataset
from tqdm.notebook import tqdm
from sklearn.metrics import matthews_corrcoef

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
np.random.seed(314)
train_full = load_dataset('winogrande', 'winogrande_xl', split='train')
train_full = train_full.shuffle()
inds = np.arange(len(train_full))
split_loc = int(len(inds)*.8)
train = train_full.select(inds[:split_loc])
val = train_full.select(inds[split_loc:])

Dataset winogrande downloaded and prepared to /root/.cache/huggingface/datasets/winogrande/winogrande_xl/1.0.0/4582c2f2a293ae12f94efa46a02a9c2156d9ce9c207a419dc18692449aae236e. Subsequent calls will reuse this data.


In [4]:
device = torch.device('cuda')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("xlnet-large-cased")
model = XLNetLMHeadModel.from_pretrained("xlnet-large-cased", mem_len=2**14).to(device)

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:211: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


In [6]:
for p in model.parameters():
    p.requires_grad = False
    p.grad = None

In [7]:
def create_pred_encodings(encodings, pred_loc):
    """ Create encodings needed for XLNet """
    encodings = deepcopy(encodings)
    encodings['input_ids'][:,pred_loc] = tokenizer.mask_token_id
    encodings['attention_mask'][:,pred_loc] = 0.

    seqlen = encodings.input_ids.size(1)
    perm_mask = torch.zeros((1, seqlen, seqlen))
    perm_mask[:,:,pred_loc[0]] = 1.0 # todo: make sure this is right
    encodings['perm_mask'] = perm_mask

    target_mapping = torch.zeros((1, 1, seqlen))
    target_mapping[:,:,pred_loc[0]] = 1.
    encodings['target_mapping'] = target_mapping

    return encodings

In [8]:
def long_seq_pass(model, encodings):
    """ Get outputs when output is greater than 1024 """
    max_len = model.base_model.mem_len
    num_passes = encodings.input_ids.size(1) // max_len + 1
    mems = None
    for i in range(num_passes):
        ran = slice(i*max_len, (i+1)*max_len)
        trunc_encodings = {
            'input_ids': encodings.input_ids[:,ran].to(device),
            'token_type_ids': encodings.token_type_ids[:,ran].to(device),
            'attention_mask': encodings.attention_mask[:,ran].to(device),
            'target_mapping': encodings.target_mapping[...,ran].to(device),
            'perm_mask': encodings.perm_mask[...,ran,ran].to(device)
        }
        if trunc_encodings['perm_mask'].bool().any():
            trunc_encodings['labels'] = encodings.labels.to(device)
        outputs = model(**trunc_encodings, mems=mems)
        mems = outputs[-1]
    return outputs

In [12]:
def get_ppl(model, cloze_sequence, target, context_examples=[]):
    """ Calculate the "perplexity" of the target subsequence given the sequence and context examples """
    # create context
    replaced_examples = []
    for ex in context_examples:
        label = int(ex['answer']) - 1
        options = [ex['option1'], ex['option2']]
        replaced_examples.append(ex['sentence'].replace('_', options[label]))
    context = ' '.join(replaced_examples)

    # create target encoding
    target_ids = tokenizer.encode(target, add_special_tokens=False)
    repl_id = tokenizer.encode('_', add_special_tokens=False)[0]
    encodings = tokenizer(context, cloze_sequence, return_tensors='pt')
    target_start = torch.where(encodings.input_ids[0] == repl_id)[0]
    target_locs = list(range(target_start, target_start + len(target_ids)))

    outputs = None
    lls = []
    for trg_loc, trg in zip(target_locs, target_ids):
        trg_loc = list(range(trg_loc, target_locs[-1] + 1))
        pred_encodings = create_pred_encodings(encodings, trg_loc).to(device)
        pred_encodings['labels'] = torch.tensor(trg).view(-1)
        outputs = long_seq_pass(model, pred_encodings)
        lls.append(outputs[0])

    return (torch.mean(torch.stack(lls)),) + outputs[1:]

In [13]:
from tqdm.notebook import tqdm

In [ ]:
num_reps = 1 # number of times to replicate the experiment
accs = {n_train: [] for n_train in [0,1,50]} # number of training examples to try
num_eval = 500

for rep in range(num_reps):
    val = val.shuffle(seed=np.random.randint(100000)) # workaround bc of seed bug in nlp
    for num_train in accs:
        print('Number of training points:', num_train)
        preds = []
        labels = []
        it = tqdm(val, miniters=5)
        for iexample, example in enumerate(it):
            context_examples = [train[int(i)] for i in np.random.choice(len(train), num_train, replace=False)]

            first_outputs = get_ppl(model, example['sentence'], example['option1'], context_examples)
            second_outputs = get_ppl(model, example['sentence'], example['option2'], context_examples)
            first_loss = first_outputs[0]
            second_loss = second_outputs[0]

            pred = torch.stack([first_loss, second_loss]).argmin()
            preds.append(pred.item())
            labels.append(int(example['answer']) - 1)

            acc = (np.array(preds) == np.array(labels)).mean()
            if iexample >= 0 and iexample % 5 == 0:
                it.set_description(f'acc: {acc*100:0.4f}%')

            if (iexample + 1) % num_eval == 0:
                break

        print(num_train, acc)
        accs[num_train].append(acc)


Number of training points: 0


0 0.602
Number of training points: 1


1 0.632
Number of training points: 50
